<a href="https://colab.research.google.com/github/rohithbojja/ANPR/blob/main/nb/Gemma3_(4B).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

Read our **[TTS Guide](https://docs.unsloth.ai/basics/text-to-speech-tts-fine-tuning)** for instructions and all our notebooks.

Read our **[Qwen3 Guide](https://docs.unsloth.ai/basics/qwen3-how-to-run-and-fine-tune)** and check out our new **[Dynamic 2.0](https://docs.unsloth.ai/basics/unsloth-dynamic-2.0-ggufs)** quants which outperforms other quantization methods!

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
# %%capture
# import os
# if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
# else:
#     # Do this only in Colab notebooks! Otherwise use pip install unsloth
    # !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    # !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    # !pip install --no-deps unsloth

### Unsloth

`FastModel` supports loading nearly any model now! This includes Vision and Text models!

In [2]:
from unsloth import FastModel
import torch

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",

    # Other popular models!
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/Llama-3.3-70B",
    "unsloth/mistral-7b-instruct-v0.3",
    "unsloth/Phi-4",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it",
    max_seq_length = 2048, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.5.7: Fast Gemma3 patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


We now add LoRA adapters so we only need to update a small amount of parameters!

In [3]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 8,           # Larger = higher accuracy, but might overfit
    lora_alpha = 8,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.language_model.model` require gradients


<a name="Data"></a>
### Data Prep
We now use the `Gemma-3` format for conversation style finetunes. We use [Maxime Labonne's FineTome-100k](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset in ShareGPT style. Gemma-3 renders multi turn conversations like below:

```
<bos><start_of_turn>user
Hello!<end_of_turn>
<start_of_turn>model
Hey there!<end_of_turn>
```

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3, phi4, qwen2.5, gemma3` and more.

In [4]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

In [5]:
!pip install -U dsad

In [6]:
!pip install -U datasets

In [7]:
from datasets import load_dataset, DatasetDict

# Load the original dataset
dataset = load_dataset("rbojja/telugu_news_summarization")

# Transformation function
def convert_to_chat(example):
    return {
        "conversations": [
            {
                "from": "human",
                "value": f"summarize: {example['text']}"
            },
            {
                "from": "gpt",
                "value": example["summary"]
            }
        ]
    }

# Apply the transformation to each split
chat_dataset = DatasetDict()
for split in dataset.keys():
    chat_dataset[split] = dataset[split].map(
        convert_to_chat,
        remove_columns=dataset[split].column_names
    )

# Sanity check
print(chat_dataset["train"][0])
dataset = chat_dataset['train']

{'conversations': [{'from': 'human', 'value': 'summarize: సాంస్కృతిక పోషకుడు సౌజన్య శీలి వైఎస్సాఆర్ మూర్తి జన్మదిన సందర్భంగా ప్రముఖ సాంస్కృతిక సంస్థలు జీవిఆర్ ఆరాధన కల్చరల్ ఫౌండేషన్, కళారాజ్యం, శిఖరం ఆర్ట్స్ థియేటర్స్, వైజ్మెన్ క్లబ్ ఆఫ్ సికింద్రాబాద్, కళానిలయం ఆధ్వర్యంలో శ్రీత్యాగరాయ గానసభలో నేడు సత్కారం నిర్వహిస్తున్నాయి. ముఖ్య అతిధిగా తెలంగాణ ప్రముఖ సలహాదారు డా. కెవి రమణ హాజరుకాగా సమాచార హక్కు చట్టం కమిషనర్ పి. విజయబాబు సాహితీవేత్త డా. పోతుకూర్చి సాంబశివరావు తదితరులు పాల్గొంటారు.'}, {'from': 'gpt', 'value': 'సాంస్కృతిక పోషకుడు సౌజన్య శీలి వైఎస్సాఆర్ మూర్తి జన్మదిన సందర్భంగా ప్రముఖ సాంస్కృతిక సంస్థలు శ్రీత్యాగరాయ గానసభలో నేడు సత్కారం నిర్వహిస్తున్నాయి. ముఖ్య అతిధి తెలంగాణ ప్రముఖ సలహాదారు డా. కెవి రమణ మరియు తదితరులు పాల్గొన్నారు.'}]}


We now use `standardize_data_formats` to try converting datasets to the correct format for finetuning purposes!

In [8]:
dataset[100]

{'conversations': [{'from': 'human',
   'value': 'summarize: పారిశుధ్య కార్మికులకు ప్రభుత్వం పెంచిన వేతన జీవోను వెంటనే విడుదల చేయాలని కోరుతూ విజయవాడ కృష్ణలంకలోని రంగా హైస్కూల్ వద్ద మున్సిపల్ కార్పొరేషన్ వర్కర్స్ యూనియన్ ( సిఐటియు ) ఆధ్వర్యంలో బుధ వారం పారిశుధ్య కార్మికులు ధర్నా నిర్వహించారు. ఈ సందర్భంగా సిఐటియు తూర్పు జోన్ 1 కార్యదర్శి వి. గురుమూర్తి మాట్లాడుతూ రాష్ట్ర ప్రభుత్వం పారిశుధ్య కార్మికులకు పెంచిన వేతనాలు మంజూరు చేయకుండా జాప్యం చేస్తోందన్నారు. సమ్మె కాలపు వేతనాన్ని కూడా ఇంతవరకు ఇవ్వలేదని తెలిపారు. ప్రభుత్వం వెంటనే జీవో విడుదల చేయకుంటే ఆందోళన తీవ్రతరం చేస్తామని హెచ్చరించారు.'},
  {'from': 'gpt',
   'value': 'రాష్ట్ర ప్రభుత్వం పారిశుధ్య కార్మికులకు పెంచిన వేతనాలు మంజూరు చేయాలనీ విజ్ఞప్తి చేస్తూ ,విజయవాడ కృష్ణలంకలోని రంగా హైస్కూల్ వద్ద మున్సిపల్ కార్పొరేషన్ వర్కర్స్ యూనియన్ ( సిఐటియు ) ఆధ్వర్యంలో బుధ వారం పారిశుధ్య కార్మికులు ధర్నా చేసారు .'}]}

In [9]:
from unsloth.chat_templates import standardize_data_formats
dataset = standardize_data_formats(dataset)

Let's see how row 100 looks like!

In [10]:
dataset[100]

{'conversations': [{'content': 'summarize: పారిశుధ్య కార్మికులకు ప్రభుత్వం పెంచిన వేతన జీవోను వెంటనే విడుదల చేయాలని కోరుతూ విజయవాడ కృష్ణలంకలోని రంగా హైస్కూల్ వద్ద మున్సిపల్ కార్పొరేషన్ వర్కర్స్ యూనియన్ ( సిఐటియు ) ఆధ్వర్యంలో బుధ వారం పారిశుధ్య కార్మికులు ధర్నా నిర్వహించారు. ఈ సందర్భంగా సిఐటియు తూర్పు జోన్ 1 కార్యదర్శి వి. గురుమూర్తి మాట్లాడుతూ రాష్ట్ర ప్రభుత్వం పారిశుధ్య కార్మికులకు పెంచిన వేతనాలు మంజూరు చేయకుండా జాప్యం చేస్తోందన్నారు. సమ్మె కాలపు వేతనాన్ని కూడా ఇంతవరకు ఇవ్వలేదని తెలిపారు. ప్రభుత్వం వెంటనే జీవో విడుదల చేయకుంటే ఆందోళన తీవ్రతరం చేస్తామని హెచ్చరించారు.',
   'role': 'user'},
  {'content': 'రాష్ట్ర ప్రభుత్వం పారిశుధ్య కార్మికులకు పెంచిన వేతనాలు మంజూరు చేయాలనీ విజ్ఞప్తి చేస్తూ ,విజయవాడ కృష్ణలంకలోని రంగా హైస్కూల్ వద్ద మున్సిపల్ కార్పొరేషన్ వర్కర్స్ యూనియన్ ( సిఐటియు ) ఆధ్వర్యంలో బుధ వారం పారిశుధ్య కార్మికులు ధర్నా చేసారు .',
   'role': 'assistant'}]}

We now have to apply the chat template for `Gemma-3` onto the conversations, and save it to `text`

In [11]:
def apply_chat_template(examples):
    texts = tokenizer.apply_chat_template(examples["conversations"])
    return { "text" : texts }
pass
dataset = dataset.map(apply_chat_template, batched = True)

Let's see how the chat template did! Notice `Gemma-3` default adds a `<bos>`!

In [12]:
dataset[100]["text"]

'<bos><start_of_turn>user\nsummarize: పారిశుధ్య కార్మికులకు ప్రభుత్వం పెంచిన వేతన జీవోను వెంటనే విడుదల చేయాలని కోరుతూ విజయవాడ కృష్ణలంకలోని రంగా హైస్కూల్ వద్ద మున్సిపల్ కార్పొరేషన్ వర్కర్స్ యూనియన్ ( సిఐటియు ) ఆధ్వర్యంలో బుధ వారం పారిశుధ్య కార్మికులు ధర్నా నిర్వహించారు. ఈ సందర్భంగా సిఐటియు తూర్పు జోన్ 1 కార్యదర్శి వి. గురుమూర్తి మాట్లాడుతూ రాష్ట్ర ప్రభుత్వం పారిశుధ్య కార్మికులకు పెంచిన వేతనాలు మంజూరు చేయకుండా జాప్యం చేస్తోందన్నారు. సమ్మె కాలపు వేతనాన్ని కూడా ఇంతవరకు ఇవ్వలేదని తెలిపారు. ప్రభుత్వం వెంటనే జీవో విడుదల చేయకుంటే ఆందోళన తీవ్రతరం చేస్తామని హెచ్చరించారు.<end_of_turn>\n<start_of_turn>model\nరాష్ట్ర ప్రభుత్వం పారిశుధ్య కార్మికులకు పెంచిన వేతనాలు మంజూరు చేయాలనీ విజ్ఞప్తి చేస్తూ ,విజయవాడ కృష్ణలంకలోని రంగా హైస్కూల్ వద్ద మున్సిపల్ కార్పొరేషన్ వర్కర్స్ యూనియన్ ( సిఐటియు ) ఆధ్వర్యంలో బుధ వారం పారిశుధ్య కార్మికులు ధర్నా చేసారు .<end_of_turn>\n'

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.

In [13]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 32,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        # max_steps = 30,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Switching to float32 training since model cannot work with float16


We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs. This helps increase accuracy of finetunes!

In [14]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

Let's verify masking the instruction part is done! Let's print the 100th row again:

In [15]:
tokenizer.decode(trainer.train_dataset[100]["input_ids"])

'<bos><bos><start_of_turn>user\nsummarize: పారిశుధ్య కార్మికులకు ప్రభుత్వం పెంచిన వేతన జీవోను వెంటనే విడుదల చేయాలని కోరుతూ విజయవాడ కృష్ణలంకలోని రంగా హైస్కూల్ వద్ద మున్సిపల్ కార్పొరేషన్ వర్కర్స్ యూనియన్ ( సిఐటియు ) ఆధ్వర్యంలో బుధ వారం పారిశుధ్య కార్మికులు ధర్నా నిర్వహించారు. ఈ సందర్భంగా సిఐటియు తూర్పు జోన్ 1 కార్యదర్శి వి. గురుమూర్తి మాట్లాడుతూ రాష్ట్ర ప్రభుత్వం పారిశుధ్య కార్మికులకు పెంచిన వేతనాలు మంజూరు చేయకుండా జాప్యం చేస్తోందన్నారు. సమ్మె కాలపు వేతనాన్ని కూడా ఇంతవరకు ఇవ్వలేదని తెలిపారు. ప్రభుత్వం వెంటనే జీవో విడుదల చేయకుంటే ఆందోళన తీవ్రతరం చేస్తామని హెచ్చరించారు.<end_of_turn>\n<start_of_turn>model\nరాష్ట్ర ప్రభుత్వం పారిశుధ్య కార్మికులకు పెంచిన వేతనాలు మంజూరు చేయాలనీ విజ్ఞప్తి చేస్తూ ,విజయవాడ కృష్ణలంకలోని రంగా హైస్కూల్ వద్ద మున్సిపల్ కార్పొరేషన్ వర్కర్స్ యూనియన్ ( సిఐటియు ) ఆధ్వర్యంలో బుధ వారం పారిశుధ్య కార్మికులు ధర్నా చేసారు .<end_of_turn>\n'

Now let's print the masked out example - you should see only the answer is present:

In [16]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[100]["labels"]]).replace(tokenizer.pad_token, " ")

'                                                                                                                                                                                        రాష్ట్ర ప్రభుత్వం పారిశుధ్య కార్మికులకు పెంచిన వేతనాలు మంజూరు చేయాలనీ విజ్ఞప్తి చేస్తూ ,విజయవాడ కృష్ణలంకలోని రంగా హైస్కూల్ వద్ద మున్సిపల్ కార్పొరేషన్ వర్కర్స్ యూనియన్ ( సిఐటియు ) ఆధ్వర్యంలో బుధ వారం పారిశుధ్య కార్మికులు ధర్నా చేసారు .<end_of_turn>\n'

In [17]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
5.57 GB of memory reserved.


Let's train the model! To resume a training run, set `trainer.train(resume_from_checkpoint = True)`

In [18]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 16,295 | Num Epochs = 1 | Total steps = 127
O^O/ \_/ \    Batch size per device = 32 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (32 x 4 x 1) = 128
 "-____-"     Trainable parameters = 14,901,248/4,000,000,000 (0.37% trained)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


OutOfMemoryError: CUDA out of memory. Tried to allocate 190.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 96.12 MiB is free. Process 111465 has 14.64 GiB memory in use. Of the allocated memory 14.27 GiB is allocated by PyTorch, and 230.76 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

<a name="Inference"></a>
### Inference
Let's run the model via Unsloth native inference! According to the `Gemma-3` team, the recommended settings for inference are `temperature = 1.0, top_p = 0.95, top_k = 64`

In [ ]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)
messages = [{
    "role": "user",
    "content": [{
        "type" : "text",
        "text" : "summarize: పారిశుధ్య కార్మికులకు ప్రభుత్వం పెంచిన వేతన జీవోను వెంటనే విడుదల చేయాలని కోరుతూ విజయవాడ కృష్ణలంకలోని రంగా హైస్కూల్ వద్ద మున్సిపల్ కార్పొరేషన్ వర్కర్స్ యూనియన్ ( సిఐటియు ) ఆధ్వర్యంలో బుధ వారం పారిశుధ్య కార్మికులు ధర్నా నిర్వహించారు. ఈ సందర్భంగా సిఐటియు తూర్పు జోన్ 1 కార్యదర్శి వి. గురుమూర్తి మాట్లాడుతూ రాష్ట్ర ప్రభుత్వం పారిశుధ్య కార్మికులకు పెంచిన వేతనాలు మంజూరు చేయకుండా జాప్యం చేస్తోందన్నారు. సమ్మె కాలపు వేతనాన్ని కూడా ఇంతవరకు ఇవ్వలేదని తెలిపారు. ప్రభుత్వం వెంటనే జీవో విడుదల చేయకుంటే ఆందోళన తీవ్రతరం చేస్తామని హెచ్చరించారు.",
    }]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)
outputs = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
)
tokenizer.batch_decode(outputs)

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "summarize: పారిశుధ్య కార్మికులకు ప్రభుత్వం పెంచిన వేతన జీవోను వెంటనే విడుదల చేయాలని కోరుతూ విజయవాడ కృష్ణలంకలోని రంగా హైస్కూల్ వద్ద మున్సిపల్ కార్పొరేషన్ వర్కర్స్ యూనియన్ ( సిఐటియు ) ఆధ్వర్యంలో బుధ వారం పారిశుధ్య కార్మికులు ధర్నా నిర్వహించారు. ఈ సందర్భంగా సిఐటియు తూర్పు జోన్ 1 కార్యదర్శి వి. గురుమూర్తి మాట్లాడుతూ రాష్ట్ర ప్రభుత్వం పారిశుధ్య కార్మికులకు పెంచిన వేతనాలు మంజూరు చేయకుండా జాప్యం చేస్తోందన్నారు. సమ్మె కాలపు వేతనాన్ని కూడా ఇంతవరకు ఇవ్వలేదని తెలిపారు. ప్రభుత్వం వెంటనే జీవో విడుదల చేయకుంటే ఆందోళన తీవ్రతరం చేస్తామని హెచ్చరించారు.",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("gemma-3")  # Local saving
tokenizer.save_pretrained("gemma-3")
# model.push_to_hub("HF_ACCOUNT/gemma-3", token = "...") # Online saving
# tokenizer.push_to_hub("HF_ACCOUNT/gemma-3", token = "...") # Online saving

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastModel
    model, tokenizer = FastModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 2048,
        load_in_4bit = True,
    )

messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "What is Gemma-3?",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

### Saving to float16 for VLLM

We also support saving to `float16` directly for deployment! We save it in the folder `gemma-3-finetune`. Set `if False` to `if True` to let it run!

In [ ]:
if False: # Change to True to save finetune!
    model.save_pretrained_merged("gemma-3-finetune", tokenizer)

If you want to upload / push to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!

In [ ]:
if False: # Change to True to upload finetune
    model.push_to_hub_merged(
        "HF_ACCOUNT/gemma-3-finetune", tokenizer,
        token = "hf_..."
    )

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now for all models! For now, you can convert easily to `Q8_0, F16 or BF16` precision. `Q4_K_M` for 4bit will come later!

In [ ]:
if False: # Change to True to save to GGUF
    model.save_pretrained_gguf(
        "gemma-3-finetune",
        quantization_type = "Q8_0", # For now only Q8_0, BF16, F16 supported
    )

Likewise, if you want to instead push to GGUF to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!

In [ ]:
if False: # Change to True to upload GGUF
    model.push_to_hub_gguf(
        "gemma-3-finetune",
        quantization_type = "Q8_0", # Only Q8_0, BF16, F16 supported
        repo_id = "HF_ACCOUNT/gemma-finetune-gguf",
        token = "hf_...",
    )

Now, use the `gemma-3-finetune.gguf` file or `gemma-3-finetune-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>


In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import evaluate
import numpy as np
import torch
from tqdm import tqdm

# Load dataset
ds = load_dataset("rbojja/telugu_news_summarization")
test_ds = ds["test"]

# Models to compare
model_names = [
    "rbojja/mt5-summarize-te_v3",
    "csebuetnlp/mT5_multilingual_XLSum"
]

# Prepare ROUGE metric
rouge = evaluate.load("rouge")

def generate_summaries(model_name, test_ds, device="cuda" if torch.cuda.is_available() else "cpu", batch_size=96):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
    preds, refs = [], []
    for i in tqdm(range(0, len(test_ds), batch_size), desc=f"Generating with {model_name}"):
        batch = test_ds[i:i+batch_size]
        inputs = tokenizer(batch["text"], return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_length=128,
                num_beams=4,
                no_repeat_ngram_size=2
            )
        decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        preds.extend(decoded_preds)
        refs.extend(batch["summary"])
    return preds, refs

results = {}
for model_name in model_names:
    preds, refs = generate_summaries(model_name, test_ds)
    # Add period if missing (for ROUGE)
    preds = [p if p.endswith((".", "!", "?", "।", "॥")) else p + "." for p in preds]
    refs = [r if r.endswith((".", "!", "?", "।", "॥")) else r + "." for r in refs]
    rouge_scores = rouge.compute(predictions=preds, references=refs)
    results[model_name] = rouge_scores

# Print results
for model_name, scores in results.items():
    print(f"\nModel: {model_name}")
    for k, v in scores.items():
        print(f"{k}: {v:.4f}")

tokenizer_config.json:   0%|          | 0.00/19.3k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/416 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/946 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

Generating with rbojja/mt5-summarize-te_v3: 100%|██████████| 22/22 [12:45<00:00, 34.80s/it]
Generating with csebuetnlp/mT5_multilingual_XLSum: 100%|██████████| 22/22 [07:02<00:00, 19.20s/it]



Model: rbojja/mt5-summarize-te_v3
rouge1: 0.1980
rouge2: 0.0500
rougeL: 0.1937
rougeLsum: 0.1944

Model: csebuetnlp/mT5_multilingual_XLSum
rouge1: 0.0563
rouge2: 0.0028
rougeL: 0.0560
rougeLsum: 0.0561


In [ ]:
from datasets import load_dataset, Dataset, load_from_disk
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import evaluate
import torch
from tqdm import tqdm

# Load dataset
ds = load_dataset("rbojja/telugu_news_summarization")
test_ds = ds["test"]

# Models to compare
model_names = [
    "rbojja/mt5-summarize-te_v3",
    "csebuetnlp/mT5_multilingual_XLSum"
    "rbojja/mt5-summarize-te_v2"
]

def generate_summaries(model_name, test_ds, device="cuda" if torch.cuda.is_available() else "cpu", batch_size=32):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
    preds = []
    for i in tqdm(range(0, len(test_ds), batch_size), desc=f"Generating with {model_name}"):
        batch = test_ds[i:i+batch_size]
        inputs = tokenizer(batch["text"], return_tensors="pt", padding=True, truncation=True, max_length=1024).to(device)
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_length=128,
                num_beams=4,
                no_repeat_ngram_size=2
            )
        decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        preds.extend(decoded_preds)
    return preds

# Generate and save after first model
first_model = model_names[0]
first_preds = generate_summaries(first_model, test_ds)
results = {
    "text": test_ds["text"],
    "ground_truth": test_ds["summary"],
    "mt5_summarize_te_v3": first_preds
}
results_ds = Dataset.from_dict(results)
results_ds.save_to_disk("telugu_news_summarization_model_comparison")
print(f"Saved predictions for {first_model}")

# Generate and save after second model
second_model = model_names[1]
second_preds = generate_summaries(second_model, test_ds)
# Load the previously saved dataset and add new column
results_ds = load_from_disk("telugu_news_summarization_model_comparison")
results_ds = results_ds.add_column("mt5_multilingual_xlsum", second_preds)
results_ds.save_to_disk("telugu_news_summarization_model_comparison")
print(f"Saved predictions for {second_model}")

# Generate and save after third model
third_model = model_names[1]
third_preds = generate_summaries(third_model, test_ds)
# Load the previously saved dataset and add new column
results_ds = load_from_disk("telugu_news_summarization_model_comparison")
results_ds = results_ds.add_column("telugu_news_summarization_model_comparison_v3", third_preds)
results_ds.save_to_disk("telugu_news_summarization_model_comparison")
print(f"Saved predictions for {second_model}")

# Calculate BLEU scores for each model
import evaluate
bleu = evaluate.load("bleu")
references = [[ref] for ref in test_ds["summary"]]
bleu_score_1 = bleu.compute(predictions=first_preds, references=references)
bleu_score_2 = bleu.compute(predictions=second_preds, references=references)
bleu_score_3 = bleu.compute(predictions=third_preds, references=references)

print(f"{first_model} BLEU: {bleu_score_1['bleu']:.4f}")
print(f"{second_model} BLEU: {bleu_score_2['bleu']:.4f}")
print(f"{third_model} BLEU: {bleu_score_3['bleu']:.4f}")

Generating with rbojja/mt5-summarize-te_v3:  56%|█████▋    | 36/64 [12:30<10:36, 22.72s/it]